# Building your Deep Neural Network: Step by Step

ඔබ විසින් පෙරදී layers දෙකක (hidden layer එකක් ඇති) neural network එකක් train කරන ආකාරය jupyter notebook එකක් ආධාරයෙන් උගෙන ඇත. මෙම mini project එකේදී ඔබ විසින් ඔබට අවශ්‍ය ඕනෑම layers ගණනක් ඇති neural network එකක් train කරන ආකාරය ඉගෙන ගනු ඇත.

- මෙම notebook එකේදී ඔබ විසින් deep neural network එකක් නිර්මාණය කිරීමට අවශ්‍ය සියලුම functions implement කරනු ඇත.
- mini project එකෙන් පසුව ඔබට මෙම functions භාවිත කර image classification සඳහා deep neural network එකක් නිර්මාණය කරන ආකාරය දැක්වෙන jupyter notebook එකක් ලබා දෙනු ලැබේ.

**මෙම activity එකෙන් පසුව ඔබට**
- ReLU වැනි non-linear units භාවිතා කර model එකක් improve කර ගන්නා ආකාරය
- hidden layers එකකට වඩා ඇති deep neural network එකක් නිර්මාණය කරගන්න ආකාරය
- භාවිතා කිරීමට පහසු neural network class එකක් නිර්මාණය කරගන්නා ආකාරය
ගැන අදහසක් ලබා ගැනීමට හැකි වනු ඇත.


**Notation**:
- Superscript $[l]$ මගින් $l$ වන layer එක සමග සම්බන්ද quantity එකක් දක්වනු ලබයි.
    - Example: $a^{[L]}$ යනු $L$ වන layer එකෙහි activation එක වේ. $W^{[L]}$ and $b^{[L]}$ යනු $L$ වන layer එකෙහි parameters වේ.
- Superscript $(i)$ මගින් $i$ වන example එක සමග සම්බන්ද quantity එකක් දක්වනු ලබයි.
    - Example: $x^{(i)}$ යනු $i$ වන training example එක වේ.
- Lowerscript $i$ මගින්, vector එකක $i$ වන entry එක දක්වනු ලබයි.
    - Example: $a^{[l]}_i$ යනු $l$ වන layer එකෙහි activation එකෙහි $i$ වන entry එක වේ


Let's get started!

In [4]:
from google.colab import drive
import sys
drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/ML/Building your Deep Neural Network - Step by Step')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 1 - Packages

මෙම පැවරුම තුළදී ඔබට අවශ්‍ය වන සියලු packages පළමුව import කරමු.
- [Numpy] (www.nphay.org) යනු පයිතන් සමඟ scientific computing සඳහා භාවිතා වන ප්‍රධාන පැකේජයයි.
- [Matplotllib] (http://matallotlib.org) යනු පයිතන් හි graph plotting සඳහා වන library එකකි.
- Dnn_utils මෙම නෝට්බුක් සඳහා අවශ්ය functions කිහිපයක් සපයන, ඔබට ලබාදෙන ලද folder එක තුල ඇති තවත් python file එකකි..
- testcase ඔබේ කාර්යයන්හි නිවැරදිබව තක්සේරු කිරීම සඳහා tests සපයයි.
- np.random.seed (1) සියලු random function calls වෙනස් නොවී තබා ගැනීමට භාවිතා කරයි. මෙම seeds වෙනස් නොකර තබා ගන්න. එසේ නොමැතිනම් ඔබට ලැබෙන පිළිතුරු බලාපොරොත්තු වන අගයට වඩා වෙනස් වනු ඇත.

In [5]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases import *
from dnn_utils import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2 - Outline of the Assignment

ඔබගේ neural network එක ගොඩනැගීම සඳහා, ඔබ "helper functions" කිහිපයක් implement කරනු ඇත. layers දෙකක neural network එකක් සහ layers L ගණනක් ඇති neural network එකක් ගොඩනැගීමට මෙම helper functions භාවිතා කල හැක. ඔබ විසින් implement කරන සෑම කුඩා helper functions එකක් සඳහාම implement කිරීමට අවශ්‍ය පියවර හරහා ඔබව ගෙන යන සවිස්තරාත්මක උපදෙස් ඇත. මෙම පැවරුමේ දළ සටහනක්ද පහතින් දක්වා ඇත, ඔබ විසින්:

-  2-layer neural network එකක් සහ L-layer neural network එකක් සඳහා අවශ්‍ය w සහ b parameters initialize කරන්න.
- Forward propagation module එක implement කරන්න.  (පහත රූපයේ දම් පාටින් පෙන්වා ඇත).
      - layer එකකට අදාල Forward propagation හි linear කොටස implement කරන්න (ප්‍රතිඵලය: $Z^{[l]}$).
      - අපි ඔබට ACTIVATION ශ්‍රිතය (relu/sigmoid) ලබා දී ඇත.
      - පෙර පියවර දෙක එක කර නව [LINEAR->ACTIVATION] function එකක් නිර්මාණය කර ගන්න.
      - [LINEAR->RELU] ඉදිරි ශ්‍රිතය L-1 වාරයක් (layer 1 සිට layer L-1 දක්වා) භාවිතා කර අවසානයේ [LINEAR->SIGMOID] එකක් එක් කරන්න (අවසාන ස්ථරය $L$ සඳහා). මෙය ඔබට නව L_model_forward ශ්‍රිතයක් ලබා දෙයි.
- cost එක ගණනය කරන්න.
- Backward Propagation Model එක ක්‍රියාත්මක කරන්න (පහත රූපයේ රතු පැහැයෙන් දක්වා ඇත).
     - layer එකක backward propagation step එකක linear කොටස සම්පූර්ණ කරන්න.
     - අපි ඔබට activation function වල අනුක්‍රමය ලබා දී ඇත. (relu_backward/sigmoid_backward)
     - පෙර පියවර දෙක නව [LINEAR->activation] backword function එකකට combine කරන්න.
     - නව L_model_backward function [LINEAR->RELU] backward propagation L-1 වරක් යොදා [LINEAR->SIGMOID] අවසාන layer එක සඳහා යොදා ගන්න.
- අවසාන වශයෙන් parameters update කරන්න.

<img src="images/final outline.png" style="width:800px;height:500px;">
<caption><center> **Figure 1**</center></caption><br>


**සටහන**\
සෑම  forward function එකක් සඳහාම අනුරූප backward function එකක් ඇත. ඔබේ forward module එකේ සෑම පියවරකදීම ඔබ යම් අගයන් cache එකක ගබඩා කරන්නේ එබැවිනි. gradients ගණනය කිරීම සඳහා cache එකෙහි ඇති අගයන් ප්‍රයෝජනවත් වේ.  backpropagation module එක තුළ ඔබ පසුව gradients ගණනය කිරීමට cache එක භාවිතා කරනු ඇත. මෙම එක් එක් පියවරයන් නිවැරදිව සිදු කරන්නේ කෙසේද යන්න මෙම පැවරුම මඟින් ඔබට පෙන්වා ඇත.

## 3 - Initialization

ඔබ ඔබේ model එක සඳහා parameters initialize කරන helper functions දෙකක් ලියන්න. 2-layer model එකක් සඳහා parameters initialize කිරීම සඳහා පළමු function එක භාවිතා කරනු ඇත. දෙවැන්න මෙම initializing ක්‍රියාවලිය $L$-layer model වලට generalize කරයි.

### 3.1 - 2-layer Neural Network

**Exercise**:\
2-layer neural network එකක parameters නිර්මාණය කිරීම සහ initialize කිරීම.

**උපදෙස්**:
- ආකෘතියේ ව්යුහය: *LINEAR -> RELU -> LINEAR -> SIGMOID*.
- weight matrices (න්‍යාස) (w) සඳහා random initialization භාවිතා කරන්න. නිවැරදි හැඩය සමඟ `np.random.randn(shape)*0.01` භාවිතා කරන්න.
- biases (b) සඳහා zero initialization භාවිතා කරන්න. `np.zeros(shape)` භාවිත කරන්න.

In [10]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer

    Returns:
    parameters -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """

    np.random.seed(1)

    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y,1))

    ### END CODE HERE ###

    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

In [11]:
parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[0.]]


**Expected output**:
       
<table style="width:80%">
  <tr>
    <td> **W1** </td>
    <td> [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]] </td>
  </tr>

  <tr>
    <td> **b1**</td>
    <td>[[ 0.]
 [ 0.]]</td>
  </tr>
  
  <tr>
    <td>**W2**</td>
    <td> [[ 0.01744812 -0.00761207]]</td>
  </tr>
  
  <tr>
    <td> **b2** </td>
    <td> [[ 0.]] </td>
  </tr>
  
</table>

### 3.2 - L-layer Neural Network

Deeper L-layer neural network එකක් initialize කිරීම වඩාත් සංකීර්ණ වන්නේ පෙර 2-layer neural network එකට වඩා තවත් බොහෝ W - weight metrices සහ b - bias vectors ඇති බැවිනි. `initialize_parameters_deep` සම්පූර්ණ කරන විට, ඔබේ එක් එක් LAYER අතර parameters වල dimansions ගැළපෙන බවට ඔබ සහතික විය යුතුය. $n^{[l]}$ යනු $l$ ස්ථරයේ ඇති node ගණන බව මතක තබා ගන්න. උදාහරණයක් ලෙස අපගේ input එක $X$ හි ප්‍රමාණය $(12288, 209)$ නම් ($m=209$ = data points  209 ක් ඇත.) එවිට:

![Alt text](image.png)

අපි python හි $W X + b$ ගණනය කරන විට python විසින් broadcasting සිදු කරන බව මතක තබා ගන්න. උදාහරණයක් ලෙස:

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$

එවිට $WX + b$ යන සුළු කිරීම පහත පරිදි වනු ඇත:

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$

\
මෙය මෙසේ සිදුවීම සඳහා,

$$ b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$

යන 1 column matrix එක, පහත ඇති 3 column matrix එක බවට broadcast වීම මගින් සකස් වී ඇත.

$$ b = \begin{bmatrix}
    s  & s  & s\\
    t  & t & t \\
    u  & u & u
\end{bmatrix}\;\;\; $$


**Exercise**: L-layer Neural Network එකක් සඳහා initialization Implement කිරීම.

**Instructions**:
- model එකෙහි structure එක *[LINEAR -> RELU] $ \times$ (L-1) -> LINEAR -> SIGMOID*. එනම්, එහි ReLU activation function එක භාවිතා කරන layers $L-1$ක් ඇති අතර, පසුව sigmoid activation function එක භාවිතා කරන output  layer එකක් ඇත.
- weight matrices සඳහා random initialization භාවිතා කරන්න. මේ සඳහා `np.random.randn(shape) * 0.01` භාවිත කරන්න.
- biases සඳහා zeros initialization භාවිතා කරන්න. මේ සඳහා `np.zeros(shape)` භාවිත කරන්න.
- විවිධ ස්ථරවල ඇති nodes ගණන ($n^{[l]}$,), `layer_dims` ලෙස ඇති variable එකක ගබඩා කර ඇත. උදාහරණයක් ලෙස, පසුගිය සතියේ ලබාදුන් 2-layer neural network එක සඳහා `layer_dims` සකස් කලහොත් එය  [2,4,1] වනු ඇත: එය input layer එකෙහි features දෙකක්, hidden layer එකෙහි nodes 4ක් සහිත එක් hidden layer එකක් සහ output layer එකෙහි 1 node එකක් සහිත neural network එකකි. මෙයින් අදහස් වන්නේ එම neural network එක `W1` හි හැඩය (4,2), `b1` (4,1), `W2` (1,4) සහ `b2` (1,1) ආකාරයේ හැඩයන් ඇති neural network එකක් බවයි. දැන් ඔබ මෙය $L$ layers වලට generalize කළ යුතුය.
- පහත දක්වා ඇත්තේ $L=1$ (1 layer neural network එකක්) සඳහා වන implementation එකයි. එය gerenarize කර L-layer neural network එකක් සඳහා implement කරන්න.

```python
    if L == 1:
        parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
        parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```

In [17]:
# GRADED FUNCTION: initialize_parameters_deep

def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network

    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """

    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        ### END CODE HERE ###

        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters

In [18]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


**Expected output**:
       
<table style="width:80%">
  <tr>
    <td> **W1** </td>
    <td>[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]</td>
  </tr>
  
  <tr>
    <td>**b1** </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]</td>
  </tr>
  
  <tr>
    <td>**W2** </td>
    <td>[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]</td>
  </tr>
  
  <tr>
    <td>**b2** </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]]</td>
  </tr>
  
</table>

## 4 - Forward propagation module

### 4.1 - Linear Forward
ඔබ දැන් parameters initialize කිරීම සිදු කර ඇත. මීලගට forward propagation module එක implement කල යුතුය. මේ සඳහා, ප්‍රථමයෙන්, අපට ඉදිරියේදීද model එක implement කිරීමේදී භාවිතා කිරීමට සිදුවන basic functions කිහිපයක් implement කරමු. මෙම functions implement කිරීම පහත අනුපිලිවෙල අනුව සිදු කරනු ලබයි.

- LINEAR
- LINEAR -> ACTIVATION (මෙහිදී ACTIVATION function එක ReLU හෝ Sigmoid වේ.)
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID (සම්පුර්ණ model එක)

linear forward module එක (සියලුම inputs(examples) එකවර forward propagate කිරීම සඳහා vectorize කර ඇති) පහත equation එක calculate කරනු ලබයි:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

මෙහි $A^{[0]} = X$ වේ.

**Exercise**: forward propagation එකෙහි linear part එක ගොඩ නගන්න.

**Reminder**:
මෙම කොටසේ mathematical representation එක වන්නේ $Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$ වේ. `np.dot()` method එක ඔබට ප්‍රයෝජනවත් වනු ඇත. නිතරම matrices වල dimensions check කර බලන්න. dimensions match නොවන අවස්ථාවලදී, `W.shape` භාවිතයෙන් W හි dimensions check කර බැලීම ඔබට උපකාරී වනු ඇත.

In [19]:
# GRADED FUNCTION: linear_forward

def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter
    cache -- a python tuple containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """

    ### START CODE HERE ### (≈ 1 line of code)
    Z = np.dot(W, A) + b
    ### END CODE HERE ###

    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)

    return Z, cache

In [20]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


**Expected output**:

<table style="width:35%">
  
  <tr>
    <td> **Z** </td>
    <td> [[ 3.26295337 -1.23429987]] </td>
  </tr>
  
</table>

### 4.2 - Linear-Activation Forward

මෙම notebook එකේදී, ඔබ විසින් activation functions දෙකක් භාවිත කරනු ඇත.

- **Sigmoid**: $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$. \
  අපි ඔබට 'sigmoid'  function එක සපයා ඇත. මෙම ශ්‍රිතය outputs **දෙකක්** return කරයි:activation value "a" සහ "Z" අඩංගු "cache" එකකින් එම outputs දෙක සමන්විත වේ.(එය අපි අනුරූප backward function එකට ඉදිරියේදී ලබා දෙනු ලබයි). ලබා දී ඇති function එක පහත ලෙස භාවිතා කිරීමට හැකිය.
  
  ```
  A, activation_cache = sigmoid(Z)
  ```

- **ReLU**: The mathematical formula for ReLu is $A = RELU(Z) = max(0, Z)$. \
  අපි ඔබට 'ReLU'  function එක සපයා ඇත. මෙම ශ්‍රිතය outputs **දෙකක්** return කරයි:activation value "a" සහ "Z" අඩංගු "cache" එකකින් එම outputs දෙක සමන්විත වේ.(එය අපි අනුරූප backward function එකට ඉදිරියේදී ලබා දෙනු ලබයි). ලබා දී ඇති function එක පහත ලෙස භාවිතා කිරීමට හැකිය.
  ```
  A, activation_cache = relu(Z)
  ```

පහසුව සඳහා, අපි functions දෙකක් (Linear සහ Activation) එක function එකකට (LINEAR->ACTIVATION) එකතු කර ගනු ලැබේ. එම නිසා ඔබ විසින් LINEAR forward step එක සහ ACTIVATION forward step දෙකම සිදු කරන එක function එකක් implement කරනු ඇත.

**Exercise**\
*LINEAR->ACTIVATION* layer එකෙහි  forward propagation implement කරන්න.මීට අදාල  Mathematical relation එක $A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})$ වේ. මෙහි activation "g" යනු sigmoid() හෝ relu() යන දෙකෙන් එකකි. linear_forward() function එක සහ නිවැරදි activation function එක භාවිතා කිරීමට සැලකිලිමත් වෙන්න..

In [21]:
# GRADED FUNCTION: linear_activation_forward

def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value
    cache -- a python tuple containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """

    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)

        ### END CODE HERE ###

    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)

        ### END CODE HERE ###

    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [22]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


**Expected output**:
       
<table style="width:35%">
  <tr>
    <td> **With sigmoid: A ** </td>
    <td > [[ 0.96890023  0.11013289]]</td>
  </tr>
  <tr>
    <td> **With ReLU: A ** </td>
    <td > [[ 3.43896131  0.        ]]</td>
  </tr>
</table>


**Note**: deep learning වලදී, "[LINEAR->ACTIVATION]" computation එක එක් layer එකක් බව මතක තබා ගන්න. මෙම පියවර දෙක මගින් layer දෙකක් නිරුපනය නොවේ.

### d) L-Layer Model

$L$-layer Neural Network එක implement කිරීමේදී තවදුරටත් පහසුව සඳහා, පෙරදී implement කරන ලද function එක භාවිතා කර  (`linear_activation_forward`, RELU activation function එක සමග) $L-1$ වාරයක්, සහ (`linear_activation_forward`, SIGMOID.activation function එක සමග) අන්තිම layer එක සඳහා යොදා ගනිමින් මෙම function එක complete කරන්න.

<img src="images/model_architecture_kiank.png" style="width:600px;height:300px;">
<caption><center> **Figure 2** : *[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* model</center></caption><br>

**Exercise**: ඉහත model එකෙහි forward propagation කොටස implement කරන්න..

**Instruction**: පහත code එකෙහි, `AL` ලෙස ඇති  variable එක මගින් $A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$ යන කොටස දැක්වේ. (මෙය සමහර අවස්ථාවල `Yhat`(Y hat) ලෙසද හැඳින්වේ. එනම් මෙම AL යනු $\hat{Y}$ වේ.)

**Tips**:
- ඔබ විසින් පෙරදී implement කරන ලද functions භාවිතා කරන්න.
- [LINEAR->RELU], (L-1) වාරයක් භාවිත කිරීමට for loop එකක් භාවිතා කරන්න.
- එක එක layer එකෙහි caches, "caches" list එකෙහි රඳවාගන්න. යම් `list` එකකට අලුත් `c` අගයක් ඇතුලත් කිරීම සඳහා ඔබට `list.append(c)` යන්න භාවිතා කල හැක.

In [23]:
# GRADED FUNCTION: L_model_forward

def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation

    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()

    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L-1 of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2

    """
    number of layers in the neural network
    """

    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A
        ### START CODE HERE ### (≈ 2 lines of code)
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation='relu')

        caches.append(cache)

        ### END CODE HERE ###

    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation='sigmoid')

    caches.append(cache)


    ### END CODE HERE ###

    assert(AL.shape == (1,X.shape[1]))

    return AL, caches

In [24]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


<table style="width:50%">
  <tr>
    <td> **AL** </td>
    <td > [[ 0.03921668  0.70498921  0.19734387  0.04728177]]</td>
  </tr>
  <tr>
    <td> **Length of caches list ** </td>
    <td > 3 </td>
  </tr>
</table>

දැන් ඔබට සම්පූර්ණ 'forward propagation' ඇති අතර එය X input එක ලබාගෙන ඔබේ predictions අඩංගු $A^{[L]}$ row vector එකක් output කරයි. එය සියලුම අතරමැදි අගයන් "caches" තුළ ගබඩා කර ගැනීමද සිදු කරයි. $A^{[L]}$ භාවිතයෙන්, ඔබට ඔබේ predictions වල cost එක ගණනය කළ හැක.

## 5 - Cost function
දැන් ඔබ විසින් forward සහ backward propagation implement කරනු ලබයි. මෙහිදී cost එක compute කිරීමට අවශ්‍ය වේ. මීට හේතු වන්නේ අපට model එක learn කරන බව දැනගැනීමට මෙම cost එක අවශ්‍ය වන බැවිනි.

**Exercise**\
cross-entropy cost එක $J$ ගණනය කරන්න. මේ සඳහා මෙම formula එක භාවිතා කරන්න.
$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$


In [25]:
# GRADED FUNCTION: compute_cost

def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """

    m = Y.shape[1]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 lines of code)
    cost = (-1/m) * np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1-Y, np.log(1-AL)))
    ### END CODE HERE ###

    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())

    return cost

In [26]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.2797765635793422


**Expected Output**:
<table>
    <tr>
    <td>**cost** </td>
    <td> 0.2797765635793422</td>
    </tr>
</table>

## 6 - Backward propagation module

forward propagation සඳහා මෙන්ම backpropagation සඳහාද ඔබ විසින් helper functions implement කල යුතුය. backpropagation භාවිතා වෙන්නේ loss function එකෙහි, parameters වලට සාපේක්ෂව gradient එක (අනුක්‍රමණය) සොයා ගැනීමට බව මතක තබා ගන්න.

**Reminder**:\
<img src="images/backprop_kiank.png" style="width:650px;height:250px;">
<caption><center> **Figure 3** : Forward සහ Backward propagation for *LINEAR->RELU->LINEAR->SIGMOID* <br> *දම් පාට blocks මගින් forward propagation represent කරන අතර, red blocks මගින් backward propagation represent කරනු ලබයි.*  </center></caption>

<!--
For those of you who are expert in calculus (you don't need to be to do this assignment), the chain rule of calculus can be used to derive the derivative of the loss $\mathcal{L}$ with respect to $z^{[1]}$ in a 2-layer network as follows:

$$\frac{d \mathcal{L}(a^{[2]},y)}{{dz^{[1]}}} = \frac{d\mathcal{L}(a^{[2]},y)}{{da^{[2]}}}\frac{{da^{[2]}}}{{dz^{[2]}}}\frac{{dz^{[2]}}}{{da^{[1]}}}\frac{{da^{[1]}}}{{dz^{[1]}}} \tag{8} $$

In order to calculate the gradient $dW^{[1]} = \frac{\partial L}{\partial W^{[1]}}$, you use the previous chain rule and you do $dW^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial W^{[1]}}$. During the backpropagation, at each step you multiply your current gradient by the gradient corresponding to the specific layer to get the gradient you wanted.

Equivalently, in order to calculate the gradient $db^{[1]} = \frac{\partial L}{\partial b^{[1]}}$, you use the previous chain rule and you do $db^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial b^{[1]}}$.

This is why we talk about **backpropagation**.
!-->

දැන් forward propagation implement කල ආකාරයට සමාන ආකාරයකින්ම, backward propagation ද පියවර 3කින් implement කරමු.
- LINEAR backward
- LINEAR -> ACTIVATION backward. මෙහිදී ACTIVATION compute කරන්නේ ReLU හෝ sigmoid activation හි derivative (අවකලනය) එක වේ.
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID backward (සම්පුර්ණ model එක)

### 6.1 - Linear backward

layer $l$ සඳහා, linear part එක වන්නේ: \
$Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ (පසුව මෙයට activation function එකක් යොදනු ලබයි.).

ඔබ දැනටමත් $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$ යන derivative එක calculate කර ඇති සිතන්න. \
ඔබට ගණනය කිරීමට අවශ්‍ය වන්නේ $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$ ලෙස සිතමු. අප lectures වලදී සාකඡචා කල කරුණු අනුව, මේ 3 ම ගණනය කිරීමට  $dZ^{[l]}$ අවශ්‍ය බව අපි දනිමු. එනම්, $dZ^{[l]}$ input එකක් ලෙස දුන් විට අපට $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$ යන තුනම ගණනය කර ගත හැක.

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">
<caption><center> **Figure 4** </center></caption>

මෙම function එකෙහි output  වන $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$ ගණනය කරනු ලබන්නේ input $dZ^{[l]}$ භාවිතයෙනි. පහත දැක්වෙන්නේ ඔබට ඒ සඳහා අවශ්‍ය වන formulas වේ.
$$ dW^{[l]} = \frac{\partial \mathcal{J} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{J} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$


**Exercise**:\
ඉහත සඳහන් formulas 3 භාවිතයෙන් linear_backward() function එක implement කරන්න.

In [31]:
# GRADED FUNCTION: linear_backward

def linear_backward(dZ, cache):
    # Here cache is "linear_cache" containing (A_prev, W, b) coming from the forward propagation in the current layer
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    dW = np.dot(dZ, A_prev.T)/m
    db = np.sum(dZ, axis=1, keepdims=True) /m
    dA_prev = np.dot(W.T, dZ)
    ### END CODE HERE ###

    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)

    return dA_prev, dW, db

In [32]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]
dW = [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
db = [[-0.14713786]
 [-0.11313155]
 [-0.13209101]]


** Expected Output**:
    
```
dA_prev =
 [[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]
dW =
 [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
db =
 [[-0.14713786]
 [-0.11313155]
 [-0.13209101]]
```

### 6.2 - Linear-Activation backward

මීළඟට ඔබ විසින්,  **`linear_backward`** සහ activation  එක සඳහා backward step එක යන දෙකම එකතු කර  **`linear_activation_backward`** නම් නව function එකක් implement කරනු ලබයි.

ඔබට මෙම `linear_activation_backward` function එක implement කිරීම පහසු කිරීමට, අප විසින් activation function වල backward step එක සපයා ඇත.

- **`sigmoid_backward`**: SIGMOID unit එකෙහි backward propagation Implementation එක මෙය වේ. පහත පරිදි ඔබය එය භාවිත කල හැකිය:

    ```python
    dZ = sigmoid_backward(dA, activation_cache)
    ```

- **`relu_backward`**: RELU unit එකෙහි backward propagation Implementation එක මෙය වේ. පහත පරිදි ඔබය එය භාවිත කල හැකිය:

    ```python
    dZ = relu_backward(dA, activation_cache)
    ```
\
\
$g(.)$ යනු activation function එක නම්,
`sigmoid_backward` සහ `relu_backward` විසින් පහත ප්ගණනය කිරීම සිදු කරයි.
$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$  

**Exercise**: \
*LINEAR->ACTIVATION* layer එක සඳහා backpropagation Implement කරන්න.

In [33]:
# GRADED FUNCTION: linear_activation_backward

def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.

    Arguments:
    dA -- post-activation gradient for current layer l
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache

    if activation == "relu":
        ### START CODE HERE ### (≈ 1 line of code)
        dZ = relu_backward(dA, activation_cache)

        ### END CODE HERE ###

    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 1 line of code)
        dZ = sigmoid_backward(dA, activation_cache)

        ### END CODE HERE ###

    ### START CODE HERE ### (≈ 1 line of code)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    ### END CODE HERE ###

    return dA_prev, dW, db

In [34]:
dAL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


**Expected output with sigmoid:**

<table style="width:100%">
  <tr>
    <td>
        dA_prev
     </td>
     <td>
         [[ 0.11017994  0.01105339]
         [ 0.09466817  0.00949723]
         [-0.05743092 -0.00576154]]
      </td>
  </tr>
    <tr>
        <td>
            dW
        </td>
        <td>
            [[ 0.10266786  0.09778551 -0.01968084]]
        </td>
  </tr>
    <tr>
        <td>
        db
       </td>
       <td >
           [[-0.05729622]]
        </td>
  </tr>
</table>

**Expected output with relu:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td>
           <td > [[ 0.44090989  0.        ]
                  [ 0.37883606  0.        ]
                  [-0.2298228   0.        ]]
            </td>

  </tr>
  
  <tr>
    <td > dW </td>
           <td > [[ 0.44513824  0.37371418 -0.10478989]] </td>
  </tr>
  
  <tr>
    <td > db </td>
           <td > [[-0.20837892]] </td>
  </tr>
</table>



### 6.3 - L-Model Backward

දැන් ඔබ මුළු network එක සඳහාම backward function එක implement කල යුතුය. පෙරදී implement කල `L_model_forward` function එකෙදි, එක් එක් iteration එකේදී, (X,W,b, and z) යන දත්ත අඩංගු cache එකක් store කරනු ලැබුවා. back propagation module එකේදී gradients ගණනය කිරීම සඳහා මෙම variables භාවිතා කරනු ලබයි. එම නිසා, `L_model_backward` function එකේදී සියලුම hidden layers හරහා iterate කල යුතුයි. මෙහිදී අප  implement කරනු ලබන්නේ back propagation බැවින් layer මත iterate කරන විට $L$ වන layer එකෙන් ආරම්භ කල යුතුය. එක් එක් layer හරහා සිදුකරන iteration වලදී එම $l$ layer එක සඳහා forward propagation වලදී store කල caches භාවිතා කිරීමට සිදු වනු ඇත. පහත Figure 5 backward pass එක සිදුවන අයුරු පෙන්වයි.


<img src="images/mn_backward.png" style="width:450px;height:300px;">
<caption><center>  **Figure 5** : Backward pass  </center></caption>

**Initializing backpropagation** :\
මෙම network එකේ output එක, $A^{[L]} = \sigma(Z^{[L]})$ වේ.\
මෙම network එක හරහා backpropagate කිරීමේදී, ඔබේ code එක,\
`dAL` $= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$
compute කිරීමට අවශ්‍ය වේ. \
ඒ සඳහා, පහත formula එක භාවිතා කරන්න. (මෙම dAL ට අදාල සමීකරණය ගොඩනැගුනු ආකාරය දැනගැනීම එතරම් වැදගත් නොවේ.)
```
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # derivative of cost with respect to AL
```

ඉහත code එක මගින් 'dAL' ගණනය කිරීමෙන් පසු, ඔබට සාමාන්‍ය ලෙස backward propagation සිදු කල හැක. Figure 5 හිදී දුටු ආකාරයට, ඔබට දැන් 'dAL' ඔබ implement කල LINEAR->SIGMOID backward function එකට ලබා දිය හැක. මෙම function එක  L_model_forward function එක මගින් store කල caches භාවිතා කරන බව නැවත මතකයට නගා ගන්න.
මෙය සිදු කිරීමෙන් පසුව, ඔබ for loop එකක් භාවිතයෙන් සියලුම අනෙක් layers හරහා LINEAR->RELU backward function එක භාවිතයෙන් iterate කල යුතුය. මෙම backward propagation වලදී ඔබ විසින් සියලුම dA, dW, සහ db අගයන් 'grads' dictionary එක තුල store කල යුතුය. එසේ store කිරීම සඳහා, පහත code එක භාවිතා කරන්න:

$$grads["dW" + str(l)] = dW^{[l]}\tag{15} $$

example: $l=3$ වන layer එක සඳහා, ඉහත code එක මගින් $dW^{[l]}$, 'grads' dictionary එකේ "dW3" key එක යටතේ store කරනු ඇත.

**Exercise**:\
*[LINEAR->RELU] $\times$ (L-1) -> [LINEAR -> SIGMOID]* model එක සඳහා backpropagation Implement කරන්න.

In [43]:
# GRADED FUNCTION: L_model_backward

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group

    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])

    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ...
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ...
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL

    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    dAL = dAL = - (np.divide(Y, AL) -np.divide(1-Y, 1-AL))
    ### END CODE HERE ###

    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)

    current_cache = caches[-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_backward(sigmoid_backward(dAL, current_cache[1]),current_cache[0])

    ### END CODE HERE ###

    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)]

        ### START CODE HERE ### (approx. 5 lines)
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_backward(sigmoid_backward(dAL, current_cache[1]), current_cache[0])
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        ### END CODE HERE ###

    return grads

In [44]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)

dW1 = [[-0.38142895 -0.05436378 -0.12122851 -0.09345065]
 [-0.36454443 -0.04886266 -0.11465667 -0.08859687]
 [-0.36758766 -0.04958047 -0.11573455 -0.08940829]]
db1 = [[0.13978379]
 [0.12259085]
 [0.12471635]]
dA1 = [[ 0.01969098 -0.12970306]
 [ 0.09890728 -0.22545732]
 [ 0.1304364  -0.31335009]
 [ 0.03215356  0.01532388]]


**Expected Output**

<table style="width:60%">
  
  <tr>
    <td > dW1 </td>
           <td > [[ 0.41010002  0.07807203  0.13798444  0.10502167]
 [ 0.          0.          0.          0.        ]
 [ 0.05283652  0.01005865  0.01777766  0.0135308 ]] </td>
  </tr>
  
  <tr>
    <td > db1 </td>
           <td > [[-0.22007063]
 [ 0.        ]
 [-0.02835349]] </td>
  </tr>
  
  <tr>
  <td > dA1 </td>
           <td > [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]] </td>

  </tr>
</table>



### 6.4 - Update Parameters

මෙම section එකේදී ඔබ විසින් gradient descent භාවිතයෙන් model එකෙහි parameters update කරනු ලබයි.

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

මෙහිදී $\alpha$ යනු learning rate එකයි. updated parameters ගණනය කිරීමෙන් අනතුරුව එම updated parameters, 'parameters' dictionary එකේ store කල යුතුය.

**Exercise**: \
`update_parameters()` function එක gradient descent භාවිතයෙන් Implement කරන්න.

**Instructions**:\
සියලුම $W^{[l]}$ සහ $b^{[l]}$ for $l = 1, 2, ..., L$ සඳහා gradient descent භාවිතයෙන් parameters update කරන්න.

In [45]:
# GRADED FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent

    Arguments:
    parameters -- python dictionary containing your parameters
    grads -- python dictionary containing your gradients, output of L_model_backward

    Returns:
    parameters -- python dictionary containing your updated parameters
                  parameters["W" + str(l)] = ...
                  parameters["b" + str(l)] = ...
    """

    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 3 lines of code)
    for l in range(L):
        parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]

    ### END CODE HERE ###
    return parameters

In [46]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


**Expected Output**:

<table style="width:100%">
    <tr>
    <td > W1 </td>
           <td > [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]] </td>
  </tr>
  
  <tr>
    <td > b1 </td>
           <td > [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]] </td>
  </tr>
  <tr>
    <td > W2 </td>
           <td > [[-0.55569196  0.0354055   1.32964895]]</td>
  </tr>
  
  <tr>
    <td > b2 </td>
           <td > [[-0.84610769]] </td>
  </tr>
</table>



## 7 - Conclusion

deep neural network එකක් ගොඩනැගීම සඳහා අවශ්ය සියලු functions implement කිරීම ගැන ඔබට සුබ පැතුම්!

මෙය සමහර විට ඔබ බලාපොරොත්තු වුවාට වැඩ තරමක් දිග පැවරුමක් වන්නට ඇත, නමුත් මෙය නිම කිරීම ඔබේ අනාගතයට වැදගත් ආයෝජනයක් වනු ඇත. මෙම පැවරුමේ අවසන් වීමෙන් පසුව මෙම functions ඔබට භාවිත කල හැකි ආකාරය පෙන්වන තවත් jupyter notebook එකක් ඔබට ඉදිරියේදී ලබා දෙනු ලැබේ.

මෙම ඊළඟ jupyter notebook එකේදී ඔබ සරල models දෙකක් තැනීමට මෙම functions භාවිතා කරනු ඇත.
- 2-layer neural network එකක්
- L-layer neural network එකක්

cat vs non-cat images classify කිරීම සඳහා ඔබ මෙම models භාවිතා කරනු ඇත!